# ETL Movies

In [12]:
print('hiworld')

hiworld


In [13]:
import pandas as pd

dfaws = pd.read_csv(r'../Datasets/movi/amazon_prime_titles.csv', sep=',', encoding='UTF-8', thousands=',', decimal='.') # importo CSV.

dfaws.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [14]:
dfaws = dfaws.drop_duplicates() #borro duplicates

In [15]:

dfaws['id'] = dfaws['show_id'].apply(lambda x: 'a' + str(x)) # agrego segun consigna la info a id column. Guardandola ahora como "id"
dfaws = dfaws.assign(plataforma='AWS') #discrimino la plataforma para luego poder diferenciar en el df maestro
dfaws = dfaws.drop('show_id', axis=1) # elimino vieja columna "show_id".
dfaws = dfaws[['id','plataforma', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']] # reordeno para mayor comodidad, reemplazando la nueva columna "id" en lugar de "show_id"

In [16]:
dfaws['rating'] = dfaws['rating'].fillna('G') #relleno nulos de rating seguun consigna con "G"

In [17]:
# De haber fechas, deberán tener el formato AAAA-mm-dd...
dfaws['date_added'] = pd.to_datetime(dfaws['date_added']) # transformo a formato date
dfaws['date_added'] = dfaws['date_added'].dt.strftime('%Y-%m-%d')  #cambio el formato de las fechas reordenando

In [18]:
dfaws = dfaws.applymap(lambda x: x.lower() if type(x) == str else x) #Transformo todo en minusculas por simplicidad y p/ evitar futuros errores.

dfaws[['duration_int', 'duration_type']] = dfaws['duration'].str.extract('(\d+) (\D+)') # Separo columna duration en duration_int (solo minutos) y duration type (seasons)
dfaws = dfaws.drop('duration', axis=1) # elimino vieja columna "duration" ya sustituida por las anteriores
dfaws = dfaws[['id','plataforma', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration_int','duration_type', 'listed_in', 'description']] # reordeno para mayor comodidad, reemplazando duration con duration_int y duration_type
# se observo en este dataframe valores faltantes en cast director country y date added (este ultimo será sustituido por el promedio del resto de las peliculas registradas en el csv, para minimizar la pérdida de datos)


In [19]:


# convierte la columna date_added a tipo datetime
dfaws['date_added'] = pd.to_datetime(dfaws['date_added'], errors='coerce')

# elige la fecha promedio y reemplaza los valores nulos en la columna date_added
date_mean = dfaws['date_added'].mean()
dfaws['date_added'].fillna(date_mean, inplace=True)

# selecciona sólo la fecha y conviértela en formato AAAA-mm-dd
dfaws['date_added'] = pd.Series(dfaws['date_added'].values).dt.strftime('%Y-%m-%d')



dfaws.fillna('unknown', inplace=True) # relleno los demas valores faltantes de country cast, director con "unknown" , tambien para minimizar la perdida de datos.


In [20]:
#transformacion de todos los tipos de datos presentes en el df.
dfaws['id'] = dfaws['id'].astype('object')
dfaws['plataforma'] = dfaws['plataforma'].astype('category')
dfaws['type'] = dfaws['type'].astype('category')
dfaws['title'] = dfaws['title'].astype('object')
dfaws['director'] = dfaws['director'].astype('object')
dfaws['cast'] = dfaws['cast'].astype('object')
dfaws['country'] = dfaws['country'].astype('category')
dfaws['date_added'] = pd.to_datetime(dfaws['date_added'])
dfaws['release_year'] = dfaws['release_year'].astype('int64')
dfaws['rating'] = dfaws['rating'].astype('category')
dfaws['duration_int'] = dfaws['duration_int'].astype('int64')
dfaws['duration_type'] = dfaws['duration_type'].astype('category')
dfaws['listed_in'] = dfaws['listed_in'].astype('category')
dfaws['description'] = dfaws['description'].astype('object')
